## Thomas Fire(2017) False Color

Author: Stephen Carroll

This project can be found at [https://github.com/stephenccodes/eds220-hwk4](https://github.com/stephenccodes/eds220-hwk4)

###  Landsat Collection 2 Level-2

 This data is a simplified collection of bands (red, green, blue, near-infrared and shortwave infrared) from the Landsat Collection 2 Level-2 atmosperically corrected surface reflectance data, collected by the Landsat 8 satellite.
The data was retrieved from the Microsof Planetary Computer data catalogue and pre-processed to remove data outside land and coarsen the spatial resolution.

Date accessed: 11/19/24

Original dats source:

[https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)

In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rioxr

# Establish a file path for the landsat data
fp = os.path.join('/courses/EDS220/data/hwk4_landsat_data/landsat8-2018-01-26-sb-simplified.nc')

# Import the landsat data
landsat = rioxr.open_rasterio(fp)

In [2]:
# Explore the data
landsat.head(3)

<xarray.Dataset> Size: 424B
Dimensions:      (band: 1, x: 3, y: 3)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 24B 1.213e+05 1.216e+05 1.218e+05
  * y            (y) float64 24B 3.952e+06 3.952e+06 3.952e+06
    spatial_ref  int64 8B 0
Data variables:
    red          (band, y, x) float64 72B ...
    green        (band, y, x) float64 72B ...
    blue         (band, y, x) float64 72B ...
    nir08        (band, y, x) float64 72B ...
    swir22       (band, y, x) float64 72B ...

In [3]:
# Find the CRS of the landsat data
landsat.rio.crs

CRS.from_epsg(32611)

In [12]:
# Find the dimensions of the landsat data
print('Shape: ', landsat.sizes)

Shape:  Frozen({'band': 1, 'x': 870, 'y': 731})


In [10]:
# Find the data type of the landsat data
print('Data type: ', landsat.values.dtype)

AttributeError: 'function' object has no attribute 'dtype'

In [14]:
print('Number of bands: ', landsat.rio.count)

InvalidDimensionOrder: Invalid dimension order. Expected order: ('band', 'y', 'x'). You can use `DataArray.transpose('band', 'y', 'x')` to reorder your dimensions.

In [13]:
print('Height: ', landsat.rio.height)
print('Width: ', landsat.rio.width)
print('Spatial bounding box: ')
print(landsat.rio.bounds(), '\n')

print('CRS', landsat.rio.crs)

InvalidDimensionOrder: Invalid dimension order. Expected order: ('band', 'y', 'x'). You can use `DataArray.transpose('band', 'y', 'x')` to reorder your dimensions.